# HPE CSI DRIVER FOR KUBERNETES 
created by Dirk Derichsweiler / k8s@hpe.com
& Stephan Koch / stephan.koch@hpe.com
/ Sept 2020

you can find this jupyter notebook here:
https://github.com/dderichswei/csi

A Container Storage Interface (CSI) Driver for Kubernetes. The HPE CSI Driver for Kubernetes allows you to use a Container Storage Provider (CSP) to perform data management operations on storage resources. The architecture of the CSI driver allows block storage vendors to implement a CSP that follows the spec (a browser friendly version).

It allows a complete separation of concerns between upstream Kubernetes core, SIG Storage (CSI owners), CSI driver author (HPE) and the backend CSP developer.


for more Information: https://scod.hpedev.io/csi_driver/

<img src="pictures/1.png" alt="SC" width="700" height="700" style="float:left">

## select the right cluster

In [91]:
ls ~/notebooks/k8sconfig

azure-dirk.derichsweiler.conf  ctc-ocp45.conf		   dahoam-rusti.conf
ctc-ddk1.conf		       ctc-rancher-demo02.conf	   hpelaptop.conf
ctc-ez-dev-cluster.conf        ctc-rancher-k3s-admin.conf  k8sconfig
ctc-ez-prod-cluster.conf       ctc-rancher-rke.conf	   tools
ctc-ez-qas-cluster.conf        dahoam-bigmama.conf


In [92]:
export KUBECONFIG=~/notebooks/k8sconfig/ctc-ocp45.conf

In [93]:
oc login -u skoch -p                                                                           

Login successful.

You have access to 60 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "simple-demo".


In [94]:
kubectl cluster-info
kubectl get nodes
#kubectl get pods -A

Kubernetes master is running at https://api.ocp.container.demo.local:6443

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
NAME                     STATUS   ROLES    AGE   VERSION
ocp-w5vxk-master-0       Ready    master   37d   v1.18.3+6c42de8
ocp-w5vxk-master-1       Ready    master   37d   v1.18.3+6c42de8
ocp-w5vxk-master-2       Ready    master   37d   v1.18.3+6c42de8
ocp-w5vxk-worker-bc5pq   Ready    worker   37d   v1.18.3+6c42de8
ocp-w5vxk-worker-gcrsm   Ready    worker   37d   v1.18.3+6c42de8
ocp-w5vxk-worker-nq4qh   Ready    worker   37d   v1.18.3+6c42de8


## Install with Helm

other installation methods: 
- Object Configuration Files (https://github.com/hpe-storage/csi-driver)
- Helm Charts (https://hub.helm.sh)
- Operator (https://operatorhub.io)

In [382]:
helm version

version.BuildInfo{Version:"v3.3.0", GitCommit:"8a4aeec08d67a7b84472007529e8097ec3742105", GitTreeState:"dirty", GoVersion:"go1.14.7"}


In [383]:
helm repo add hpe-storage https://hpe-storage.github.io/co-deployments/

"hpe-storage" has been added to your repositories


In [563]:
helm repo update
helm search repo hpe-csi-driver

Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "hpe-storage" chart repository
Update Complete. ⎈ Happy Helming!⎈ 
NAME                      	CHART VERSION	APP VERSION	DESCRIPTION                                       
hpe-storage/hpe-csi-driver	1.3.0        	1.3.0      	A Helm chart for installing the HPE CSI Driver ...


### Helm Parameters

|Parameter|	Description	|Default|
|:--------|:-----------:|:------|
|backendType	|Name of the HPE Storage backend type (nimble, hpe3parprimera)|	nimble|
|secret.create	|Enabled creation of secret along with driver deployment	|true|
|secret.backend	|HPE storage backend hostname or IP address.	|192.168.1.1|
|secret.username|	Username for the backend.|	admin|
|secret.password|	Password for the backend.	|admin|
|crd.nodeInfo.create|	Create nodeinfo CRDs required by HPE CSI driver. Should only enable with HELM 2, as they are automatically created with HELM 3 without this flag.|	false|
|crd.volumeInfo.create|	Create volumeinfo CRDs required by HPE CSI driver for 3PAR Primera. Should only enable with HELM 2, as they are automatically created with HELM 3 without this flag.|	false|
|logLevel|	Log level. Can be one of info, debug, trace, warn and error|	info|
|imagePullPolicy	|Image pull policy (Always, IfNotPresent, Never).	|Always|
|storageClass.name	|The name to assign the created StorageClass.|	hpe-standard|
|storageClass.create|	Enables creation of StorageClass to consume this hpe-csi-driver instance	|true|
|storageClass.defaultClass|	Whether to set the created StorageClass as the clusters default StorageClass.	|false|
|storageClass.parameters.fsType|	Type of file system being used (ext4, ext3, xfs, btrfs)	|xfs|
|storageClass.parameters.volumeDescription|	Volume description for volumes created using HPE CSI driver|	-|
|storageClass.parameters.accessProtocol|	Access protocol to use for storage connectivity (iscsi, fc)	|iscsi|
|storageClass.parameters.provisioningType|	Provisioning type for HPE 3PAR Primera	|tpvv|
|storageClass.parameters.cpg|	CPG type for HPE 3PAR Primera	|FC_r6|

In [597]:
helm install hpe-csi hpe-storage/hpe-csi-driver --namespace kube-system --set secret.backend=10.0.32.253 --set secret.username=dderichswei --set secret.password=Compaq1!
#helm uninstall hpe-csi --namespace kube-system

NAME: hpe-csi
LAST DEPLOYED: Thu Sep 10 10:59:28 2020
NAMESPACE: kube-system
STATUS: deployed
REVISION: 1
TEST SUITE: None


**For FibreChannel Access:**

Above we installed for the iscsi protocol.
For FC use:

``helm install hpe-csi hpe-storage/hpe-csi-driver --namespace kube-system --set storageClass.parameters.accessProtocol=fc --set secret.backend=<nimbleip> --set secret.username=<user> --set secret.password=<PW>``

and do manually the FC Zoning.

Adjust the accessProtocol for the StorageClass from iscsi to fc

``accessProtocol: fc``

In [600]:
#from version 1.3 onwards no default sc will be created! check values.yaml
kubectl get sc
echo "---"
kubectl get sc hpe-standard -o yaml

NAME                PROVISIONER        RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
ddnimble            csi.hpe.com        Delete          Immediate           true                   67m
default (default)   com.mapr.csi-kdf   Delete          Immediate           false                  16d
mapr-platinum-sc    com.mapr.csi-kdf   Delete          Immediate           false                  14d
---
Error from server (NotFound): storageclasses.storage.k8s.io "hpe-standard" not found


: 1

In [592]:
kubectl get sc hpe-standard -o yaml

Error from server (NotFound): storageclasses.storage.k8s.io "hpe-standard" not found


: 1

# create Secret

nimble-secret is created with them "Helm Installation"

In [408]:
kubectl get secret nimble-secret -n kube-system -o yaml
kubectl get secret

Error from server (NotFound): secrets "nimble-secret" not found
No resources found in simple-demo namespace.


## create own Storageclass

A StorageClass is used to provision or clone an HPE Nimble Storage-backed persistent volume. It can also be used to import an existing HPE Nimble Storage volume or clone of a snapshot into the Kubernetes cluster. The parameters are grouped below by those same workflows.

``These are optional parameters unless specified.``

In [409]:
kubectl get sc
#kubectl get pods -A

NAME                PROVISIONER        RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
default (default)   com.mapr.csi-kdf   Delete          Immediate           false                  16d
mapr-platinum-sc    com.mapr.csi-kdf   Delete          Immediate           false                  14d


### StorageClass parameters
#### Common parameters for provisioning and cloning

|Parameter|	String	|Description|
|:--------|:-------:|:----------|
|accessProtocol|	Text	|The access protocol to use when accessing the persistent volume ("fc" or "iscsi"). Defaults to "iscsi" when unspecified.|
|destroyOnDelete|	Boolean	|Indicates the backing Nimble volume (including snapshots) should be destroyed when the PVC is deleted.|
|limitIops|	Integer	|The IOPS limit of the volume. The IOPS limit should be in the range 256 to 4294967294, or -1 for unlimited (default).|
|limitMbps|	Integer|	The MB/s throughput limit for the volume.|
|description|	Text|	Text to be added to the volume's description on the Nimble array.|
|performancePolicy|	Text|	The name of the performance policy to assign to the volume. Default example performance policies include "Backup Repository", "Exchange 2003 data store", "Exchange 2007 data store", "Exchange 2010 data store", "Exchange log", "Oracle OLTP", "Other Workloads", "SharePoint", "SQL Server", "SQL Server 2012", "SQL Server Logs".|
|protectionTemplate|	Text	|The name of the protection template to assign to the volume. Default examples of protection templates include "Retain-30Daily", "Retain-48Hourly-30aily-52Weekly", and "Retain-90Daily".|
|folder	|Text	|The name of the Nimble folder in which to place the volume.|
|thick	|Boolean	|Indicates that the volume should be thick provisioned.|
|dedupeEnabled|	Boolean	|Indicates that the volume should enable deduplication.|
|syncOnDetach|	Boolean	|Indicates that a snapshot of the volume should be synced to the replication partner each time it is detached from a node.|

#### Provisioning parameters
These parameters are immutable for clones once a volume has been created.

``fsOwner, fsMode, and fsCreateOptions are not applicable when using volumeMode: Block in the PersistentVolumeClaim.``

|Parameter	|String	|Description|
|:----------|:-----:|:----------|
|fsOwner	|userId:groupId	|The user id and group id that should own the root directory of the filesystem.|
|fsMode	|Octal digits|	1 to 4 octal digits that represent the file mode to be applied to the root directory of the filesystem.|
|fsCreateOptions|	Text	|A string to be passed to the mkfs command. These flags are opaque to CSI and are therefore not validated. To protect the node, only the following characters are allowed: [a-zA-Z0-9=, \-].|
|encrypted|	Boolean	|Indicates that the volume should be encrypted.|
|pool	|Text|	The name of the pool in which to place the volume.|

#### Pod inline volume parameters (Local Ephemeral Volumes)
These parameters are applicable only for Pod inline volumes and to be specified within Pod spec.

``All parameters are required for inline ephemeral volumes.``

|Parameter	|String	|Description|
|:----------|:-----:|:----------|
|csi.storage.k8s.io/ephemeral	|Boolean	|Indicates that the request is for ephemeral inline volume. This is a mandatory parameter and must be set to "true".|
|inline-volume-secret-name|	Text	|A reference to the secret object containing sensitive information to pass to the CSI driver to complete the CSI NodePublishVolume call.|
|inline-volume-secret-namespace	|Text	|The namespace of inline-volume-secret-name for ephemeral inline volume.|
|size	|Text	|The size of ephemeral volume specified in MiB or GiB. If unspecified, a default value will be used.|
|accessProtocol|	Text	|Storage access protocol to use, "iscsi" or "fc".|

#### Cloning parameters

Cloning supports two modes of cloning. Either use cloneOf and reference a PVC in the current namespace or use importVolAsClone and reference a Nimble volume name to clone and import to Kubernetes.

|Parameter	|String	|Description|
|:----------|:-----:|:----------|
|cloneOf	|Text	|The name of the PV to be cloned. cloneOf and importVolAsClone are mutually exclusive.|
|importVolAsClone|	Text	|The name of the Nimble volume to clone and import. importVolAsClone and cloneOf are mutually exclusive.|
|snapshot|	Text	|The name of the snapshot to base the clone on. This is optional. If not specified, a new snapshot is created.|
|createSnapshot|	Boolean	|Indicates that a new snapshot of the volume should be taken matching the name provided in the snapshot parameter. If the snapshot parameter is not specified, a default name will be created.|



#### Import parameters

Importing volumes to Kubernetes requires the source Nimble volume to be offline. In case of reverse replication, the upstream volume should be in offline state. All previous Access Control Records and Initiator Groups will be stripped from the volume when put under control of the HPE CSI Driver.

|Parameter	|String	|Description|
|:----------|:-----:|:----------|
|importVolumeName	|Text	|The name of the Nimble volume to import.|
|snapshot|	Text|	The name of the Nimble snapshot to restore the imported volume to after takeover. If not specified, the volume will not be restored.|
|takeover|	Boolean	|Indicates the current group will takeover ownership of the Nimble volume and volume collection. This should be performed against a downstream replica.|
|reverseReplication	|Boolean	|Reverses the replication direction so that writes to the Nimble volume are replicated back to the group where it was replicated from.|
|forceImport	|Boolean	|Forces the import of a volume that is not owned by the group and is not part of a volume collection. If the volume is part of a volume collection, use takeover instead.|

#### VolumeSnapshotClass parameters

These parametes are for VolumeSnapshotClass objects when using CSI snapshots. Please see using CSI snapshots for more details.

|Parameter	|String	|Description|
|:----------|:-----:|:----------|
|description	|Text	|Text to be added to the snapshot's description on the Nimble array.|
|writable	|Boolean	|Indicates if the snapshot is writable on the Nimble array.|
|online	|Boolean	|Indicates if the snapshot is set to online on the Nimble array.|

# Make container persistent with the HPE CSI Interface

<img src="pictures/2.png" alt="SC" width="700" height="700" style="float:left">

## check and set the environment

In [4]:
kubectl get nodes
echo "--- "
kubectl get sc
echo "--- "
kubectl config get-contexts

NAME                     STATUS                     ROLES    AGE   VERSION
ocp-w5vxk-master-0       Ready                      master   36d   v1.18.3+6c42de8
ocp-w5vxk-master-1       Ready,SchedulingDisabled   master   36d   v1.18.3+6c42de8
ocp-w5vxk-master-2       Ready                      master   36d   v1.18.3+6c42de8
ocp-w5vxk-worker-bc5pq   Ready                      worker   36d   v1.18.3+6c42de8
ocp-w5vxk-worker-gcrsm   Ready,SchedulingDisabled   worker   36d   v1.18.3+6c42de8
ocp-w5vxk-worker-nq4qh   Ready                      worker   36d   v1.18.3+6c42de8
--- 
NAME             PROVISIONER                    RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
ctc-k8s-n4       csi.hpe.com                    Delete          Immediate           true                   23d
ddnimble         csi.hpe.com                    Delete          Immediate           true                   29h
hpe-standard     csi.hpe.com                    Delete          Immediate           true 

In [5]:
kubectl create ns simple-demo
kubectl config set-context --current --namespace=simple-demo
kubectl get all

Error from server (AlreadyExists): namespaces "simple-demo" already exists
Context "simple-demo/api-ocp-container-demo-local:6443/skoch@demo.local" modified.
No resources found in simple-demo namespace.


## create a Storageclass


<img src="pictures/sc.png" alt="SC" width="350" height="350" style="float:left">

In [95]:
#Storageclass
cat << 'EOF' | kubectl apply -f -
---
kind: StorageClass
apiVersion: storage.k8s.io/v1
metadata:
  name: ddnimble
provisioner: csi.hpe.com
allowVolumeExpansion: true
parameters:
  accessProtocol: iscsi
  csi.storage.k8s.io/controller-expand-secret-name: nimble-ctc-k8s-n4
  csi.storage.k8s.io/controller-expand-secret-namespace: hpe-csi-driver
  csi.storage.k8s.io/controller-publish-secret-name: nimble-ctc-k8s-n4
  csi.storage.k8s.io/controller-publish-secret-namespace: hpe-csi-driver
  csi.storage.k8s.io/fstype: xfs
  csi.storage.k8s.io/node-publish-secret-name: nimble-ctc-k8s-n4
  csi.storage.k8s.io/node-publish-secret-namespace: hpe-csi-driver
  csi.storage.k8s.io/node-stage-secret-name: nimble-ctc-k8s-n4
  csi.storage.k8s.io/node-stage-secret-namespace: hpe-csi-driver
  csi.storage.k8s.io/provisioner-secret-name: nimble-ctc-k8s-n4
  csi.storage.k8s.io/provisioner-secret-namespace: hpe-csi-driver
  description: Volume created by the HPE CSI Driver for Kubernetes
  folder: "simple-demo"
  reclaimPolicy: Delete
  volumeBindingMode: Immediate
  destroyOnDelete: "true"
  limitIops: "38400"
  limitMbps: "2048"
  allowOverrides: description, limitIops, limitMbps, folder, destroyOnDelete
  
EOF

storageclass.storage.k8s.io/ddnimble unchanged


In [97]:
kubectl get sc
echo "---"
#kubectl get sc ddnimble -o yaml

NAME             PROVISIONER                    RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
ctc-k8s-n4       csi.hpe.com                    Delete          Immediate           true                   24d
ddnimble         csi.hpe.com                    Delete          Immediate           true                   2d1h
ddnimble-rwx     csi.hpe.com                    Delete          Immediate           true                   19h
hpe-standard     csi.hpe.com                    Delete          Immediate           true                   24d
thin (default)   kubernetes.io/vsphere-volume   Delete          Immediate           false                  37d
---


## create Persistent Volume Claim (User request)


<img src="pictures/pvc.png" alt="SC" width="350" height="350" style="float:left">

In [98]:
cat << 'EOF' | kubectl apply -f -
---
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: ddpvc1
  annotations:
    csi.hpe.com/description: "demo the nimble plugin for K8S/Openshift/Docker/Rancher"
    csi.hpe.com/limitIOPS: "8000"
    csi.hpe.com/destroyOnDelete: "true" 
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 5Gi
  storageClassName: ddnimble
EOF

persistentvolumeclaim/ddpvc1 created


In [99]:
#kubectl get ns
#echo "---"
kubectl get pvc
echo "---"
kubectl get pv
echo "---"
kubectl describe pvc ddpvc1
echo "---"
kubectl describe pv 

NAME     STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
ddpvc1   Bound    pvc-48a299ea-5f68-4083-9290-1fca9f5086d1   5Gi        RWO            ddnimble       7s
---
NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                                         STORAGECLASS   REASON   AGE
pvc-11b5f81d-04e7-4399-8d11-5c74df27b0e1   100Gi      RWO            Delete           Bound    openshift-image-registry/image-registry-pvc   thin                    21d
pvc-48a299ea-5f68-4083-9290-1fca9f5086d1   5Gi        RWO            Delete           Bound    simple-demo/ddpvc1                            ddnimble                5s
---
Name:          ddpvc1
Namespace:     simple-demo
StorageClass:  ddnimble
Status:        Bound
Volume:        pvc-48a299ea-5f68-4083-9290-1fca9f5086d1
Labels:        <none>
Annotations:   csi.hpe.com/description: demo the nimble plugin for K8S/Openshift/Docker/Rancher
       

## create Pod and attach Persistent Volume Claim

<img src="pictures/podpvc.png" alt="SC" width="350" height="350" style="float:left">

In [100]:
cat << 'EOF' | kubectl apply -f -
---
kind: Pod
apiVersion: v1
metadata:
  name: ddpod1
spec:
  containers:
  - name: nginx
    image: nginx
    volumeMounts:
    - name: export
      mountPath: /export
  restartPolicy: Always
  volumes:
  - name: export
    persistentVolumeClaim:
      claimName: ddpvc1
EOF

pod/ddpod1 created


In [112]:
kubectl get pod
#echo "---"
#kubectl describe pod ddpod1

NAME     READY   STATUS    RESTARTS   AGE
ddpod1   1/1     Running   0          39s


## let´s have a look inside of the container

In [113]:
kubectl exec pod/ddpod1 -- ls /export

## change some files inside the container

In [114]:
kubectl exec ddpod1 -- bash -c "date > /export/ddpod1.txt" 
kubectl exec ddpod1 -- cat /export/ddpod1.txt

Thu Sep 17 14:48:52 UTC 2020


## destroy the pod1, create an second, with volume of first pod

<img src="pictures/delpod.png" alt="SC" width="550" height="550" style="float:left">

In [115]:
kubectl delete pod ddpod1

pod "ddpod1" deleted


In [116]:
kubectl get pod

No resources found in simple-demo namespace.


In [117]:
kubectl get pv

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                                         STORAGECLASS   REASON   AGE
pvc-11b5f81d-04e7-4399-8d11-5c74df27b0e1   100Gi      RWO            Delete           Bound    openshift-image-registry/image-registry-pvc   thin                    21d
pvc-48a299ea-5f68-4083-9290-1fca9f5086d1   5Gi        RWO            Delete           Bound    simple-demo/ddpvc1                            ddnimble                117s


## create a new Pod

In [118]:
cat << 'EOF' | kubectl apply -f -
---
kind: Pod
apiVersion: v1
metadata:
  name: ddpod2
spec:
  containers:
  - name: nginx
    image: nginx
    volumeMounts:
    - name: export
      mountPath: /export
  restartPolicy: Always
  volumes:
  - name: export
    persistentVolumeClaim:
      claimName: ddpvc1
EOF

pod/ddpod2 created


In [125]:
kubectl get pod
echo "---"
#kubectl describe pod ddpod2

NAME     READY   STATUS    RESTARTS   AGE
ddpod2   1/1     Running   0          37s
---


In [127]:
kubectl exec ddpod2 -- ls /export
kubectl exec ddpod2 -- cat /export/ddpod1.txt
kubectl exec ddpod2 -- bash -c "date > /export/ddpod2.txt" 
kubectl exec ddpod2 -- cat /export/ddpod2.txt
kubectl exec ddpod2 -- sync;sync

ddpod1.txt
Thu Sep 17 14:48:52 UTC 2020
Thu Sep 17 14:50:27 UTC 2020


## clone the volume

<img src="pictures/clone.png" alt="SC" width="550" height="550" style="float:left">

In [128]:
cat << 'EOF' | kubectl create -f -
---
  kind: PersistentVolumeClaim
  apiVersion: v1
  metadata:
    name: ddpvc1-clone
  spec:
    storageClassName: ddnimble
    dataSource:
      name: ddpvc1
      kind: PersistentVolumeClaim
    accessModes:
      - ReadWriteOnce
    resources:
      requests:
        storage: 5Gi
EOF


persistentvolumeclaim/ddpvc1-clone created


In [129]:
kubectl get pvc
echo "---"
kubectl get pv
echo "---"
kubectl describe pvc ddpvc1-clone

NAME           STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
ddpvc1         Bound    pvc-48a299ea-5f68-4083-9290-1fca9f5086d1   5Gi        RWO            ddnimble       3m57s
ddpvc1-clone   Bound    pvc-3fbd1492-28c3-415e-bbe9-a7fe1a125c2a   5Gi        RWO            ddnimble       4s
---
NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                                         STORAGECLASS   REASON   AGE
pvc-11b5f81d-04e7-4399-8d11-5c74df27b0e1   100Gi      RWO            Delete           Bound    openshift-image-registry/image-registry-pvc   thin                    21d
pvc-3fbd1492-28c3-415e-bbe9-a7fe1a125c2a   5Gi        RWO            Delete           Bound    simple-demo/ddpvc1-clone                      ddnimble                3s
pvc-48a299ea-5f68-4083-9290-1fca9f5086d1   5Gi        RWO            Delete           Bound    simple-demo/ddpvc1                            ddnimble    

In [130]:
kubectl create -f - << EOF
---
kind: Pod
apiVersion: v1
metadata:
  name: ddpod3
spec:
  containers:
  - name: nginx
    image: nginx
    volumeMounts:
    - name: export
      mountPath: /export
  restartPolicy: Always
  volumes:
  - name: export
    persistentVolumeClaim:
      claimName: ddpvc1-clone
EOF

pod/ddpod3 created


In [133]:
kubectl get pod
echo "---"
kubectl get pvc
echo "---"
kubectl describe pod ddpod3

NAME     READY   STATUS    RESTARTS   AGE
ddpod2   1/1     Running   0          2m16s
ddpod3   1/1     Running   0          12s
---
NAME           STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
ddpvc1         Bound    pvc-48a299ea-5f68-4083-9290-1fca9f5086d1   5Gi        RWO            ddnimble       4m29s
ddpvc1-clone   Bound    pvc-3fbd1492-28c3-415e-bbe9-a7fe1a125c2a   5Gi        RWO            ddnimble       36s
---
Name:         ddpod3
Namespace:    simple-demo
Priority:     0
Node:         ocp-w5vxk-worker-nq4qh/10.1.36.184
Start Time:   Thu, 17 Sep 2020 14:51:36 +0000
Labels:       <none>
Annotations:  k8s.v1.cni.cncf.io/network-status:
                [{
                    "name": "openshift-sdn",
                    "interface": "eth0",
                    "ips": [
                        "10.128.2.22"
                    ],
                    "default": true,
                    "dns": {}
                }]
              k8s

In [134]:
kubectl exec ddpod3 -- ls -l /export
kubectl exec ddpod3 -- cat /export/ddpod2.txt

total 8
-rw-r--r--. 1 root root 29 Sep 17 14:48 ddpod1.txt
-rw-r--r--. 1 root root 29 Sep 17 14:50 ddpod2.txt
Thu Sep 17 14:50:27 UTC 2020


## snapshot the volume

https://scod.hpedev.io/csi_driver/using.html#enabling_csi_snapshots

### Deploy the necessary resources

As per the Kubernetes Special Interest Group (SIG) Storage, the snapshot controllers, custom resource definitions and RBAC resources should be deployed on the cluster by the vendor of the Kubernetes distribution, not the CSI driver vendor. These resources are not deployed on upstream Kubernetes 1.17.4, which is being used in this tutorial. Now, let’s deploy the necessary resources.

**IMPORTANT:before, let´s check if we do have the right api version is already enabled, as in v1.17 we need v1beta1 as the endpoint for the snapshot.**

In [48]:
kubectl api-versions|grep snap

snapshot.storage.k8s.io/v1beta1


if required you can exchange the create command to delete, to get the newest crd version.

In [49]:
#some resources may exist already
kubectl create -f https://raw.githubusercontent.com/kubernetes-csi/external-snapshotter/master/client/config/crd/snapshot.storage.k8s.io_volumesnapshotclasses.yaml
kubectl create -f https://raw.githubusercontent.com/kubernetes-csi/external-snapshotter/master/client/config/crd/snapshot.storage.k8s.io_volumesnapshotcontents.yaml
kubectl create -f https://raw.githubusercontent.com/kubernetes-csi/external-snapshotter/master/client/config/crd/snapshot.storage.k8s.io_volumesnapshots.yaml
#kubectl apply -f https://raw.githubusercontent.com/kubernetes-csi/external-snapshotter/master/config/crd/snapshot.storage.k8s.io_volumesnapshotclasses.yaml
#kubectl apply -f https://raw.githubusercontent.com/kubernetes-csi/external-snapshotter/master/config/crd/snapshot.storage.k8s.io_volumesnapshotcontents.yaml
#kubectl apply -f https://raw.githubusercontent.com/kubernetes-csi/external-snapshotter/master/config/crd/snapshot.storage.k8s.io_volumesnapshots.yaml

kubectl apply -f https://raw.githubusercontent.com/kubernetes-csi/external-snapshotter/master/deploy/kubernetes/snapshot-controller/rbac-snapshot-controller.yaml
kubectl apply -f https://raw.githubusercontent.com/kubernetes-csi/external-snapshotter/master/deploy/kubernetes/snapshot-controller/setup-snapshot-controller.yaml

Error from server (AlreadyExists): error when creating "https://raw.githubusercontent.com/kubernetes-csi/external-snapshotter/master/client/config/crd/snapshot.storage.k8s.io_volumesnapshotclasses.yaml": customresourcedefinitions.apiextensions.k8s.io "volumesnapshotclasses.snapshot.storage.k8s.io" already exists
Error from server (AlreadyExists): error when creating "https://raw.githubusercontent.com/kubernetes-csi/external-snapshotter/master/client/config/crd/snapshot.storage.k8s.io_volumesnapshotcontents.yaml": customresourcedefinitions.apiextensions.k8s.io "volumesnapshotcontents.snapshot.storage.k8s.io" already exists
Error from server (AlreadyExists): error when creating "https://raw.githubusercontent.com/kubernetes-csi/external-snapshotter/master/client/config/crd/snapshot.storage.k8s.io_volumesnapshots.yaml": customresourcedefinitions.apiextensions.k8s.io "volumesnapshots.snapshot.storage.k8s.io" already exists
serviceaccount/snapshot-controller unchanged
clusterrole.rbac.author

**IMPORTANT**

``as we are depolying not in the default namespace, we had to tweak the RBAC Clusterrolebinding for the serviceAccount (snapshot-controller).``

In [50]:
kubectl patch ClusterRolebinding snapshot-controller-role -p '{"subjects":[{"kind":"ServiceAccount","name":"snapshot-controller","namespace":"simple-demo"}]}'

clusterrolebinding.rbac.authorization.k8s.io/snapshot-controller-role patched


In [40]:
kubectl get ClusterRoleBinding snapshot-controller-role -o json  # namespace is patched from default to simple-demo in this case.

{
    "apiVersion": "rbac.authorization.k8s.io/v1",
    "kind": "ClusterRoleBinding",
    "metadata": {
        "annotations": {
            "kubectl.kubernetes.io/last-applied-configuration": "{\"apiVersion\":\"rbac.authorization.k8s.io/v1\",\"kind\":\"ClusterRoleBinding\",\"metadata\":{\"annotations\":{},\"name\":\"snapshot-controller-role\"},\"roleRef\":{\"apiGroup\":\"rbac.authorization.k8s.io\",\"kind\":\"ClusterRole\",\"name\":\"snapshot-controller-runner\"},\"subjects\":[{\"kind\":\"ServiceAccount\",\"name\":\"snapshot-controller\",\"namespace\":\"default\"}]}\n"
        },
        "creationTimestamp": "2020-09-13T17:20:30Z",
        "managedFields": [
            {
                "apiVersion": "rbac.authorization.k8s.io/v1",
                "fieldsType": "FieldsV1",
                "fieldsV1": {
                    "f:metadata": {
                        "f:annotations": {
                            ".": {},
                            "f:kubectl.kubernetes.io/last-applied-co

### create the VolumeSnapshotClass

<img src="pictures/snapshoting.png" alt="SC" width="550" height="550" style="float:left">

For each CSI driver that supports snapshots, at least one VolumeSnapshotClass object needs to be created. There’s only one backend that supports snapshots on this cluster and the VolumeSnapshotClass is therefore marked as default, which makes it easy for users to not care about implementation details.

In [60]:
kubectl create -f - << EOF
---
apiVersion: snapshot.storage.k8s.io/v1beta1 
kind: VolumeSnapshotClass
metadata:
  name: hpe-snapshot
  annotations:
    snapshot.storage.kubernetes.io/is-default-class: "true"
driver: csi.hpe.com
deletionPolicy: Delete
parameters:
  description: "Snapshot created by the HPE CSI Driver"
  csi.storage.k8s.io/snapshotter-secret-name: nimble-ctc-k8s-n4
  csi.storage.k8s.io/snapshotter-secret-namespace: hpe-csi-driver
EOF

volumesnapshotclass.snapshot.storage.k8s.io "hpe-snapshot" deleted


In [52]:
kubectl get VolumeSnapshotClass
kubectl get VolumeSnapshotClass -o yaml

NAME           DRIVER        DELETIONPOLICY   AGE
hpe-snapshot   csi.hpe.com   Delete           4s
apiVersion: v1
items:
- apiVersion: snapshot.storage.k8s.io/v1beta1
  deletionPolicy: Delete
  driver: csi.hpe.com
  kind: VolumeSnapshotClass
  metadata:
    annotations:
      snapshot.storage.kubernetes.io/is-default-class: "true"
    creationTimestamp: "2020-09-15T13:25:27Z"
    generation: 1
    managedFields:
    - apiVersion: snapshot.storage.k8s.io/v1beta1
      fieldsType: FieldsV1
      fieldsV1:
        f:deletionPolicy: {}
        f:driver: {}
        f:metadata:
          f:annotations:
            .: {}
            f:snapshot.storage.kubernetes.io/is-default-class: {}
        f:parameters:
          .: {}
          f:csi.storage.k8s.io/snapshotter-secret-name: {}
          f:csi.storage.k8s.io/snapshotter-secret-namespace: {}
          f:description: {}
      manager: kubectl-create
      operation: Update
      time: "2020-09-15T13:25:27Z"
    name: hpe-snapshot
    resourc

### create Snapshot (User request)

<img src="pictures/snapshot.png" alt="SC" width="950" height="950" style="float:left">

In [53]:
cat << 'EOF' | kubectl create -f -
---
apiVersion: snapshot.storage.k8s.io/v1beta1
kind: VolumeSnapshot
metadata:
  name: ddpvc-clone-snap1
spec:
  source:
    persistentVolumeClaimName: "ddpvc1-clone"
EOF

volumesnapshot.snapshot.storage.k8s.io/ddpvc-clone-snap1 created


In [56]:
kubectl get volumesnapshot

No resources found in simple-demo namespace.


In [55]:
kubectl delete volumesnapshot ddpvc-clone-snap1

volumesnapshot.snapshot.storage.k8s.io "ddpvc-clone-snap1" deleted


## expand an Volume

<img src="pictures/expand.png" alt="SC" width="350" height="350" style="float:left">

To perform expansion operations on Kubernetes 1.14+, you must enhance your StorageClass with some additional attributes.

``kubectl patch sc ddnimble -p '{ "allowVolumeExpansion": true }'``

In [9]:
kubectl describe sc ddnimble 

Name:            ddnimble
IsDefaultClass:  No
Annotations:     kubectl.kubernetes.io/last-applied-configuration={"allowVolumeExpansion":true,"apiVersion":"storage.k8s.io/v1","kind":"StorageClass","metadata":{"annotations":{},"name":"ddnimble"},"parameters":{"accessProtocol":"iscsi","allowOverrides":"description, limitIops, limitMbps, folder, destroyOnDelete","csi.storage.k8s.io/controller-expand-secret-name":"nimble-ctc-k8s-n4","csi.storage.k8s.io/controller-expand-secret-namespace":"hpe-csi-driver","csi.storage.k8s.io/controller-publish-secret-name":"nimble-ctc-k8s-n4","csi.storage.k8s.io/controller-publish-secret-namespace":"hpe-csi-driver","csi.storage.k8s.io/fstype":"xfs","csi.storage.k8s.io/node-publish-secret-name":"nimble-ctc-k8s-n4","csi.storage.k8s.io/node-publish-secret-namespace":"hpe-csi-driver","csi.storage.k8s.io/node-stage-secret-name":"nimble-ctc-k8s-n4","csi.storage.k8s.io/node-stage-secret-namespace":"hpe-csi-driver","csi.storage.k8s.io/provisioner-secret-name":"nimbl

In [57]:
kubectl exec -it ddpod2 -- bash -c 'df -h /export'

Filesystem          Size  Used Avail Use% Mounted on
/dev/mapper/mpathc  5.0G   68M  5.0G   2% /export


In [58]:
cat << 'EOF' | kubectl apply -f -
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: ddpvc1
  annotations:
    csi.hpe.com/description: "demo the nimble plugin for K8S/Openshift/Docker/Rancher"
    csi.hpe.com/limitIOPS: "8000"
    csi.hpe.com/destroyOnDelete: "true" 
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 16Gi
  storageClassName: ddnimble  
EOF


persistentvolumeclaim/ddpvc1 configured


In [59]:
# IMPORTANT: it may take a while until the new size is applied (est.: 1 Minute)
kubectl exec -it ddpod2 -- bash -c 'df -h /export'

Filesystem          Size  Used Avail Use% Mounted on
/dev/mapper/mpathc   16G  148M   16G   1% /export


In [16]:
kubectl get pv,pvc

NAME                                                        CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                                         STORAGECLASS   REASON   AGE
persistentvolume/pvc-11b5f81d-04e7-4399-8d11-5c74df27b0e1   100Gi      RWO            Delete           Bound    openshift-image-registry/image-registry-pvc   thin                    17d
persistentvolume/pvc-606e4744-07c8-4721-af53-8985f1c22901   16Gi       RWO            Delete           Bound    simple-demo/ddpvc1                            ddnimble                20m
persistentvolume/pvc-6e3687c5-836e-4112-8cd3-177bb60d31b1   5Gi        RWO            Delete           Bound    hpe-csi-driver/ddpvc1a                        ctc-k8s-n4              20d
persistentvolume/pvc-bd0e1cf6-4cfc-4369-a904-e67c033e1f5b   5Gi        RWO            Delete           Bound    simple-demo/ddpvc1-clone                      ddnimble                17m

NAME                                 STATUS   VOLUME                 

## import an pre-existing Volume 
(Volume exists on Nimble, but unknown to k8s)

In [370]:
cat << 'EOF' | kubectl apply -f -
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: import-nimble-volume
provisioner: csi.hpe.com
parameters:
  description: "Volume provisioned by the HPE CSI Driver"
  accessProtocol: "iscsi"
  csi.storage.k8s.io/fstype: xfs
  csi.storage.k8s.io/provisioner-secret-name: nimble-secret
  csi.storage.k8s.io/provisioner-secret-namespace: kube-system
  csi.storage.k8s.io/controller-publish-secret-name: nimble-secret
  csi.storage.k8s.io/controller-publish-secret-namespace: kube-system
  csi.storage.k8s.io/node-stage-secret-name: nimble-secret
  csi.storage.k8s.io/node-stage-secret-namespace: kube-system
  csi.storage.k8s.io/node-publish-secret-name: nimble-secret
  csi.storage.k8s.io/node-publish-secret-namespace: kube-system
  csi.storage.k8s.io/controller-expand-secret-name: nimble-secret
  csi.storage.k8s.io/controller-expand-secret-namespace: kube-system
  destroyOnDelete: "true"
  # importVolAsClone: "VolumetoImport"
  # importVolumeName: "VolumetoImport"
# Extras...|
  allowOverrides: limitIops, limitMbps, folder, importVolAsClone, importVolumeName
#allowVolumeExpansion: True
EOF

storageclass.storage.k8s.io/import-nimble-volume created


In [371]:
kubectl get sc

NAME                   PROVISIONER        RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
default (default)      com.mapr.csi-kdf   Delete          Immediate           false                  15d
hpe-standard           csi.hpe.com        Delete          Immediate           true                   23h
import-nimble-volume   csi.hpe.com        Delete          Immediate           false                  4s
mapr-platinum-sc       com.mapr.csi-kdf   Delete          Immediate           false                  13d


important:
**allowOverrides: limitIops, limitMbps, folder, importVolAsClone, importVolumeName**

``Import note: if you use the importVolumeName, the name of the volume will be renamed
form myDatabase to pvc-<uid>``

``myimportedvolume   Bound    pvc-66b42c29-dcdc-413e-8a28-c147be4aa1d9   5Gi        RWO            import-nimble-volume   6s``

In [ ]:
cat << 'EOF' | kubectl apply -f -
---
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: myimportedvolume
  annotations:  
    csi.hpe.com/importVolAsClone: "myDatabase"
    csi.hpe.com/description: "myDatabase Demo Volume"
    # csi.hpe.com/importVolumeName: "myDatabase"
    csi.hpe.com/folder: "simple-demo"
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 5Gi
  storageClassName: import-nimble-volume
EOF

In [293]:
kubectl get pvc

No resources found in simple-demo namespace.


In [ ]:
kubectl create -f - << EOF
---
kind: Pod
apiVersion: v1
metadata:
  name: ddpod4
spec:
  containers:
  - name: nginx
    image: nginx
    volumeMounts:
    - name: export
      mountPath: /export
  restartPolicy: Always
  volumes:
  - name: export
    persistentVolumeClaim:
      claimName: myimportedvolume
EOF

In [294]:
kubectl get pods

NAME                             READY   STATUS        RESTARTS   AGE
ddpod2-deploy-68c7c6c7bc-72wns   0/1     Terminating   0          5h24m
ddpod2-deploy-68c7c6c7bc-gkhs2   0/1     Terminating   0          5h24m


In [295]:
kubectl exec -it ddpod4 ls /export

kubectl exec [POD] [COMMAND] is DEPRECATED and will be removed in a future version. Use kubectl kubectl exec [POD] -- [COMMAND] instead.
Error from server (NotFound): pods "ddpod4" not found


: 1

# RWX Access

with CSI 1.30 RWX access was enabled by setting up an NFS Server for the volumes inside Kubernetes
RWX use case:
For shared filesystems where multiple Pods in the same Namespace need simultaneous access to a PVC.


Using the NFS Server Provisioner
Enabling the NFS Server Provisioner to allow RWX and ROX access mode for a PVC is straightforward. Create a new StorageClass and set .parameters.nfsResources to "true". Any subsequent claim to the StorageClass will create a NFS server Deployment on the cluster with the associated objects running on top of a RWO PVC.

Any RWO claim made against the StorageClass will also create a NFS server Deployment. This allows diverse connectivity options among the Kubernetes worker nodes as the HPE CSI Driver will look for nodes labelled csi.hpe.com/hpe-nfs=true before submitting the workload for scheduling. This allows dedicated NFS worker nodes without user workloads using taints and tolerations.

By default, the NFS Server Provisioner deploy resources in the "hpe-nfs" Namespace. This makes it easy to manage and diagnose. However, to use CSI data management capabilities on the PVCs, the NFS resources need to be deployed in the same Namespace as the RWX/ROX requesting PVC. This is controlled by the nfsNamespace StorageClass parameter. See base StorageClass parameters for more information.


The NFS Server Provisioner is not enabled by the default StorageClass and needs a custom StorageClass. The following sections are tailored to help understand the NFS Server Provisioner capabilities.

* Using the NFS Server Provisioner
* NFS Server Provisioner StorageClass parameters
* Diagnosing the NFS Server Provisioner issues

In [296]:
kubens simple-demo

Context "kubernetes-admin@k8s-1" modified.
Active namespace is "simple-demo".


## create an new Storageclass for RWX access

In [84]:
#Storageclass
cat << 'EOF' | kubectl create -f -
---
kind: StorageClass
apiVersion: storage.k8s.io/v1
metadata:
  name: ddnimble-rwx
provisioner: csi.hpe.com
parameters:
  accessProtocol: iscsi
  csi.storage.k8s.io/controller-expand-secret-name: nimble-ctc-k8s-n4
  csi.storage.k8s.io/controller-expand-secret-namespace: hpe-csi-driver
  csi.storage.k8s.io/controller-publish-secret-name: nimble-ctc-k8s-n4
  csi.storage.k8s.io/controller-publish-secret-namespace: hpe-csi-driver
  csi.storage.k8s.io/fstype: xfs
  csi.storage.k8s.io/node-publish-secret-name: nimble-ctc-k8s-n4
  csi.storage.k8s.io/node-publish-secret-namespace: hpe-csi-driver
  csi.storage.k8s.io/node-stage-secret-name: nimble-ctc-k8s-n4
  csi.storage.k8s.io/node-stage-secret-namespace: hpe-csi-driver
  csi.storage.k8s.io/provisioner-secret-name: nimble-ctc-k8s-n4
  csi.storage.k8s.io/provisioner-secret-namespace: hpe-csi-driver
  description: Volume created by the HPE CSI Driver for Kubernetes
  folder: "simple-demo"
  reclaimPolicy: Delete
  volumeBindingMode: Immediate
  destroyOnDelete: "true"
  limitIops: "38400"
  limitMbps: "2048"
  allowOverrides: description, limitIops, limitMbps, folder, destroyOnDelete
  fsMode: "0777"
  nfsResources: "true"                            ### SETUP nfs
  nfsNamespace: simple-demo                       ### where will the vols reside
allowVolumeExpansion: true
EOF


storageclass.storage.k8s.io/ddnimble-rwx created


In [25]:
kubectl get sc

NAME             PROVISIONER                    RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
ctc-k8s-n4       csi.hpe.com                    Delete          Immediate           true                   23d
ddnimble         csi.hpe.com                    Delete          Immediate           true                   29h
ddnimble-rwx     csi.hpe.com                    Delete          Immediate           true                   4s
hpe-standard     csi.hpe.com                    Delete          Immediate           true                   23d
thin (default)   kubernetes.io/vsphere-volume   Delete          Immediate           false                  36d


## create Pods through an deployment and attach Persistent Volume Claim

In [85]:
cat << 'EOF' | kubectl create -f -
---
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: ddpvc2-rwx
  annotations:
    csi.hpe.com/description: "demo the nimble plugin for K8S/Openshift/Docker/Rancher"
    csi.hpe.com/limitIOPS: "8000"
    csi.hpe.com/destroyOnDelete: "true" # volume will not be deleted
spec:
  accessModes:
    -  ReadWriteMany           ##### SET RWX ACCESS 
  resources:
    requests:
      storage: 6Gi
  storageClassName: ddnimble-rwx
EOF

persistentvolumeclaim/ddpvc2-rwx created


In [34]:
kubectl get pvc,pv,pod

NAME                                                                 STATUS    VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
persistentvolumeclaim/ddpvc2-rwx                                     Pending                                                                        ddnimble-rwx   2m20s
persistentvolumeclaim/hpe-nfs-9f679baf-0717-4a70-b986-e6f0b5f7e73f   Bound     pvc-b9d55d04-f337-4417-908f-a2081ebae129   6Gi        RWO            ddnimble-rwx   2m20s

NAME                                                        CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                                                      STORAGECLASS   REASON   AGE
persistentvolume/pvc-11b5f81d-04e7-4399-8d11-5c74df27b0e1   100Gi      RWO            Delete           Bound    openshift-image-registry/image-registry-pvc                thin                    20d
persistentvolume/pvc-b9d55d04-f337-4417-908f-a2081ebae129   6Gi        RWO            Delete    

In [21]:
kubectl get persistentvolumeclaim/hpe-nfs-c3a79441-a4f3-4ec3-ab5e-9ce4f0850741  -o yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  annotations:
    csi.hpe.com/description: demo the nimble plugin for K8S/Openshift/Docker/Rancher
    csi.hpe.com/destroyOnDelete: "true"
    csi.hpe.com/limitIOPS: "8000"
    csi.hpe.com/nfsPVC: "true"
    pv.kubernetes.io/bind-completed: "yes"
    pv.kubernetes.io/bound-by-controller: "yes"
    volume.beta.kubernetes.io/storage-provisioner: csi.hpe.com
  creationTimestamp: "2020-09-13T17:37:32Z"
  finalizers:
  - kubernetes.io/pvc-protection
  managedFields:
  - apiVersion: v1
    fieldsType: FieldsV1
    fieldsV1:
      f:metadata:
        f:annotations:
          .: {}
          f:csi.hpe.com/description: {}
          f:csi.hpe.com/destroyOnDelete: {}
          f:csi.hpe.com/limitIOPS: {}
          f:csi.hpe.com/nfsPVC: {}
          f:volume.beta.kubernetes.io/storage-provisioner: {}
        f:finalizers:
          .: {}
          v:"kubernetes.io/pvc-protection": {}
      f:spec:
        f:accessModes: {}
        f:resources:


In [35]:
kubectl get volumeattachment

No resources found


In [86]:
cat << 'EOF' | kubectl create -f -
---
kind: Deployment
apiVersion: apps/v1
metadata:
  name: ddpod2-deploy
spec:
  replicas: 2
  selector:
    matchLabels:
      app: ddpod-rwx
  template:
    metadata:
      labels:
        app: ddpod-rwx
    spec:
      containers:
      - name: nginx
        image: nginxinc/nginx-unprivileged
        #image: nginx
        volumeMounts:
        - name: export
          mountPath: /export
      restartPolicy: Always
      volumes:
      - name: export
        persistentVolumeClaim:
          claimName: ddpvc2-rwx
EOF

deployment.apps/ddpod2-deploy created


In [67]:
kubectl get pod -o wide

NAME                                                            READY   STATUS    RESTARTS   AGE   IP            NODE                     NOMINATED NODE   READINESS GATES
ddpod1                                                          1/1     Running   0          19m   10.128.2.8    ocp-w5vxk-worker-nq4qh   <none>           <none>
ddpod2-deploy-6648cf48cc-8s8dc                                  1/1     Running   0          16m   10.128.2.10   ocp-w5vxk-worker-nq4qh   <none>           <none>
ddpod2-deploy-6648cf48cc-pjlz7                                  1/1     Running   0          16m   10.128.2.9    ocp-w5vxk-worker-nq4qh   <none>           <none>
hpe-nfs-2f3b2a9e-0820-441f-a6e1-7cff81804d51-59f4d6c476-bbzlf   1/1     Running   0          22m   10.128.2.5    ocp-w5vxk-worker-nq4qh   <none>           <none>


In [237]:
kubectl get pod/ddpod2-deploy-68c7c6c7bc-gkhs2 -o json

{
    "apiVersion": "v1",
    "kind": "Pod",
    "metadata": {
        "annotations": {
            "cni.projectcalico.org/podIP": "10.192.0.149/32",
            "cni.projectcalico.org/podIPs": "10.192.0.149/32",
            "kubernetes.io/psp": "hcp-psp-privileged"
        },
        "creationTimestamp": "2020-09-04T07:47:31Z",
        "deletionGracePeriodSeconds": 30,
        "deletionTimestamp": "2020-09-04T10:09:29Z",
        "generateName": "ddpod2-deploy-68c7c6c7bc-",
        "labels": {
            "app": "ddpod-rwx",
            "pod-template-hash": "68c7c6c7bc"
        },
        "name": "ddpod2-deploy-68c7c6c7bc-gkhs2",
        "namespace": "simple-demo",
        "ownerReferences": [
            {
                "apiVersion": "apps/v1",
                "blockOwnerDeletion": true,
                "controller": true,
                "kind": "ReplicaSet",
                "name": "ddpod2-deploy-68c7c6c7bc",
                "uid": "29d077bc-f517-4133-ae15-e4bd76ce1f36"
          

## change some files from accessing BOTH containers## 

In [87]:
POD1=ddpod2-deploy-6648cf48cc-9hs7x 
POD2=ddpod2-deploy-6648cf48cc-xxn6j

#write someting from POD1
kubectl exec -it pod/${POD1} -- bash -c "df -h  /export"
kubectl exec -it pod/${POD1} -- ls /export
kubectl exec pod/${POD1} -- bash -c "date > /export/ddpod1.txt" 
kubectl exec pod/${POD1} -- cat /export/ddpod1.txt
# access from POD2
kubectl exec -it pod/${POD2} -- bash -c "df -h  /export"
kubectl exec -it pod/${POD2} -- ls /export
kubectl exec pod/${POD2} -- cat /export/ddpod1.txt
kubectl exec pod/${POD2} -- bash -c "date >> /export/ddpod1.txt" 
echo "from POD2"
kubectl exec pod/${POD2} -- cat /export/ddpod1.txt
# from POD1
echo "from POD1"
kubectl exec pod/${POD1} -- cat /export/ddpod1.txt

Filesystem            Size  Used Avail Use% Mounted on
172.30.40.32:/export  6.0G   75M  6.0G   2% /export
x
Wed Sep 16 19:42:48 UTC 2020
Filesystem            Size  Used Avail Use% Mounted on
172.30.40.32:/export  6.0G   75M  6.0G   2% /export
ddpod1.txt  x
Wed Sep 16 19:42:48 UTC 2020
from POD2
Wed Sep 16 19:42:48 UTC 2020
Wed Sep 16 19:42:49 UTC 2020
from POD1
Wed Sep 16 19:42:48 UTC 2020
Wed Sep 16 19:42:49 UTC 2020


In [304]:
kubectl delete pod ddpod2-deploy-68c7c6c7bc-72wns ddpod2-deploy-68c7c6c7bc-gkhs2

pod "ddpod2-deploy-68c7c6c7bc-72wns" deleted
pod "ddpod2-deploy-68c7c6c7bc-gkhs2" deleted



In [333]:
kubectl get pod

NAME                             READY   STATUS        RESTARTS   AGE
ddpod2-deploy-68c7c6c7bc-72wns   0/1     Terminating   0          5h58m
ddpod2-deploy-68c7c6c7bc-gkhs2   0/1     Terminating   0          5h58m


## cleanup

In [375]:
kubectl delete deployment.apps/ddpod2-deploy
kubectl delete pod ddpod2 ddpod3 ddpod4 snapshot-controller-0
kubectl delete sc ddnimble
kubectl delete sc import-nimble-volume
kubectl delete volumesnapshot ddpvc-clone-snap1
kubectl delete volumesnapshotclass hpe-snapshot
kubectl delete ns simple-demo
#kubectl delete pv $(kubectl get pv| grep -i simple-demo| awk ' { print $1 }')

Error from server (NotFound): pods "ddpod2" not found
Error from server (NotFound): pods "ddpod3" not found
Error from server (NotFound): pods "ddpod4" not found
Error from server (NotFound): pods "snapshot-controller-0" not found
Error from server (NotFound): storageclasses.storage.k8s.io "ddnimble" not found
storageclass.storage.k8s.io "import-nimble-volume" deleted
Error from server (NotFound): volumesnapshots.snapshot.storage.k8s.io "ddpvc-clone-snap1" not found
Error from server (NotFound): volumesnapshotclasses.snapshot.storage.k8s.io "hpe-snapshot" not found
Error from server (NotFound): namespaces "simple-demo" not found


: 1

In [376]:
kubectl get namespaces

NAME                      STATUS   AGE
customer-demo             Active   13d
default                   Active   15d
dev-tenant1               Active   15d
hpe-csi                   Active   15d
hpe-externalclusterinfo   Active   15d
hpecp                     Active   15d
hpecp-bootstrap           Active   15d
kd-apps                   Active   15d
kd-mlops                  Active   15d
kube-node-lease           Active   15d
kube-public               Active   15d
kube-system               Active   15d
kubernetes-dashboard      Active   15d
prod-tenant1              Active   15d
